In [1]:
import pandas as pd

In [2]:
import requests
domain = "ajc.com"

def get_pagerank(domain):
    headers = {'API-OPR':'4ok88wgckg8o0cgcswo4gc4kkc0sgocw0woww4o0'}
    url = 'https://openpagerank.com/api/v1.0/getPageRank?domains%5B0%5D=' + domain
    request = requests.get(url, headers=headers)
    if request.status_code == 200:
        result = request.json()
        if result['status_code'] == 200:
            pr_int = result['response'][0]['page_rank_integer']
            pr_dec = result['response'][0]['page_rank_decimal']
            rank = result['response'][0]['rank']

            return pr_int, pr_dec, rank



In [3]:

import whois

def whois_checker(domain):
    w = whois.whois(domain)
    domain_registrar = w.get('registrar')
    postal_code = w.get('registrant_postal_code')
    country = w.get('country')

    return  domain_registrar, postal_code, country



In [4]:
import virustotal_python
from pprint import pprint
from base64 import urlsafe_b64encode
import json


def virus_scan(domain):
    with virustotal_python.Virustotal("242305dedab935c6a39736d0749ae88f2b68f1e2f0c8e82c81d8955c5fd194d3") as vtotal:
        resp = vtotal.request(f"domains/{domain}")
        virus_data = resp.json()['data']['attributes']['last_analysis_stats']
        return virus_data['harmless'], virus_data['malicious'], virus_data['suspicious'], virus_data['undetected'], virus_data['timeout']





In [7]:
from requests_html import HTMLSession
from collections import Counter
from urllib.parse import urlparse

#try except outside this func
def link_checker(domain):
    session = HTMLSession()

    r = session.get("https://"+domain)
    unique_netlocs = Counter(urlparse(link).netloc for link in r.html.absolute_links)
    outbound_n = 0
    local_n = 0
    links = []
    for link in unique_netlocs:
        # print(link, unique_netlocs[link])
        links.append(link)
        if domain.lower() in link:
            local_n += unique_netlocs[link]
        else:
            outbound_n += unique_netlocs[link]

    return local_n, outbound_n, local_n + outbound_n, links




In [8]:
links

['subscribe.ajc.com',
 'www.ajc.com',
 'ajc.zendesk.com',
 'legislativenavigator.ajc.com',
 'twitter.com',
 'events.ajc.com',
 'classifieds.ajc.com',
 'live.ajc.com',
 'editions.ajc.com',
 'facebook.com',
 'doctors.ajc.com',
 'myaccount.ajc.com',
 'play.google.com',
 'ajc.com',
 'www.facebook.com',
 'www.pinterest.com',
 'www.nieonline.com',
 'apps.apple.com',
 'jobs.coxenterprises.com',
 'secure.qgiv.com',
 'www.reddit.com',
 'games.ajc.com']

In [27]:


from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

Options = Options()
Options.headless = True 

domain = 'ralblaw.com'
def check_login(domain, driver):
    wp_xpath = "//a[starts-with(@href, 'https://wordpress.org')]"
    try:
        displayed = driver.find_element('xpath', wp_xpath).is_displayed()

        return displayed
    except:

        return False

def check_page_source(domain, driver):
    try:
        driver.get(f"https://{domain}/wp-admin")
        wp_pg = "wp-content" in driver.page_source
        return wp_pg
    except:

        return False

def check_wp(domain):

    service_obj = Service(executable_path="C:/Users/cvaal/chromedriver.exe")
    driver = webdriver.Chrome(options=Options, service=service_obj)
    driver.implicitly_wait(1)
    driver.get(f"https://{domain}/wp-admin")
    login = check_login(domain, driver)
    pg_source = check_page_source(domain, driver)
    wp = None
    if login == False and pg_source == False:
        wp = False
    else:
        wp = True

    driver.close()
    return wp



   



In [30]:
with open('FakeNews_URLs.txt', 'r') as f:
    fakenews_sites = f.read().split('\n')
    fakenews_sites = [site.strip() for site in fakenews_sites]


def get_site_data(domain):
    if requests.head('http://'+domain, allow_redirects=True).status_code == 200:
        pr_int, pr_dec, rank = get_pagerank(domain)
        domain_registrar, postal_code, country = whois_checker(domain)
        harmless, malicious, suspicious, undetected, timeout = virus_scan(domain)
        local_n, outbound_links, total_links, links = link_checker(domain)
        wp_check = check_wp(domain)

        return domain, pr_int, pr_dec, rank, domain_registrar, postal_code, country, harmless, malicious, suspicious, undetected, timeout, local_n, outbound_links, total_links, links, wp_check 
    else:
        return 'Site down'

# domain = 'reddit.com'

columns = ['Domain', 'Page rank integer', 'Page rank decimal', 'Site Rank', 'Domain registrar', 'Postal code', 'Country of origin', 'Harmless', 'Malicious', 'Suspicious', 'Undetected', 'Timeout', 'No. of Local links', 'No. of Outbound links', 'Total links', 'Links to fake news site?', 'Wordpress?']

site_df_dict = {}

for i, domain in enumerate(fakenews_sites):
    if i == 3:
        break
    
    site_data = get_site_data(domain)
    if type(site_data) == tuple:
        for column, data in zip(columns, site_data):
            if type(data) != list:
                site_df_dict[column] = data
            else:
                if any(site in data for site in fakenews_sites):
                    site_df_dict[column] = True
                else:
                    site_df_dict[column] = False

In [31]:
site_data

'Site down'

265